# SITCOM-989 - M1M3 Inertia Compensation Performance

We need plots and metrics to evaluate the performance of the M1M3 Inertia Compensation System as described in [SITCOM-989].    
Examples of plots are:

* Hardpoint Load Cell Forces Minima and Maxima during slews as a function of time.
* Correlate the plots above with accelerations, velocities, and positions.
* (any other ideas?)

Petr asked to analyse the data obtained when slewing the telescope around 80 deg in elevation with and without inertia forces. 
The two datasets below that he used as an example contain movement from -100 deg in azimuth to 100 deg in a single slew. 
On both cases, we are using 30% motion settings in azimuth. 

* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]
* [M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]


[SITCOM-989]: https://jira.lsstcorp.org/browse/SITCOM-989


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-08-02 22:02 - 2023-08-02 22:04 UTC]: https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Zhttps://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-08-02T20%3A00%3A00.000Z&upper=2023-08-03T02%3A00%3A00.000Z&zoomedLower=2023-08-02T22%3A02%3A24.799Z&zoomedUpper=2023-08-02T22%3A04%3A02.450Z


[M1M3 TMA Inertial forces Chronograph Dashboard on 2023-07-28 02:15 - 2023-07-28 02:17 UTC]:https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/252?redirect=%2Flogin%3Fredirect%3D%252Fsources%252F1%252Fdashboards%252F252%253Frefresh%253D30s%2526tempVars%255BDownsample%255D%253DDefault%2526tempVars%255BFunction%255D%253Draw%2526lower%253Dnow%2528%2529%252520-%25252015m%2526zoomedLower%253D2023-08-02T21%25253A23%25253A19.366Z%2526zoomedUpper%253D2023-08-02T21%25253A23%25253A23.843Z&refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=mean%28%29&lower=2023-07-28T02%3A00%3A00.000Z&upper=2023-07-28T03%3A30%3A00.000Z&zoomedLower=2023-07-28T02%3A15%3A45.730Z&zoomedUpper=2023-07-28T02%3A17%3A11.966Z

## Notebook Preparation

In [ ]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.time import Time, TimeDelta
from dataclasses import dataclass

from lsst.sitcom import vandv
from lsst.sitcom.summit.utils.tmaUtils import TMA

In [ ]:
efd_client = vandv.efd.create_efd_client()

In [ ]:
number_of_hardpoints = 6
measured_forces_topics = [f"measuredForce{i}" for i in range(6)]

### Helper Functions and Classes

#### SlewDataSet

In [ ]:
@dataclass
class SlewDataSet:
    """
    Hold all the data associated with a slew.
    
    Attributes
    ----------
    name : str
        Human readable name
    start : `astropy.time.Time`
        Start date and time in ISO format and in UTC.
    end : `astropy.time.Time`
        End date and time in ISO format and in UTC.
    df : pd.DataFrame | None = None
        Pandas DataFrame containg relevant data. 
    """
    name : str
    start : Time
    end : Time
    df : pd.DataFrame | None = None

#### query_dataset

In [ ]:
async def query_dataset(start, end):
    """
    Queries all the relevant data, resample them to have the same requency and merge them in a single dataframe.
    
    Parameters
    ----------
    start : str
        Start date and time in ISO format and in UTC.
    end : str 
        End date and time in ISO format and in UTC.
        
    Returns
    -------
    pd.DataFrame
    """    
    hp_measured_forces = await efd_client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData", 
        measured_forces_topics, 
        Time(start, scale='utc'), 
        Time(end, scale='utc')
    )
    hp_measured_forces = hp_measured_forces.resample("1s").mean()
    
    return hp_measured_forces

#### get_stats

In [ ]:
def get_minmax(s):
    """
    Calculate minimum, maximum, and peak-to-peak values for two halves of a given pandas Series.

    Parameters
    ----------
    s : pandas.Series
        The input pandas Series containing data.

    Returns
    -------
    pandas.Series
        A Series containing the following calculated values for the two halves of the input Series:
        - begin_min: Minimum value of the first half of the Series.
        - begin_max: Maximum value of the first half of the Series.
        - begin_ptp: Peak-to-peak (ptp) value of the first half of the Series (abs(max - min)).
        - end_min: Minimum value of the second half of the Series.
        - end_max: Maximum value of the second half of the Series.
        - end_ptp: Peak-to-peak (ptp) value of the second half of the Series (abs(max - min)).

    Notes
    -----
    This function divides the input Series into two halves and calculates the minimum, maximum, and peak-to-peak values
    for each half. The peak-to-peak value is the absolute difference between the maximum and minimum values.

    Example
    -------
    >>> import pandas as pd
    >>> data = [1, 2, 3, 4, 5, 6, 7, 8]
    >>> series = pd.Series(data, name='example')
    >>> minmax_values = get_minmax(series)
    """

    half_timestamp = len(s.index) // 2
    first_half = s[:s.index[half_timestamp]]
    second_half = s[s.index[half_timestamp:]]

    first_half_ptp = abs(first_half.max() - first_half.min())
    second_half_ptp = abs(second_half.max() - second_half.min())

    result = pd.Series(
        data=[
            first_half.min(),
            first_half.max(),
            first_half_ptp,
            second_half.min(),
            second_half.max(),
            second_half_ptp,
        ],
        index=[
            "begin_min",
            "begin_max",
            "begin_ptp",
            "end_min",
            "end_max",
            "end_ptp",
        ],
        name=s.name,
    )

    return result

#### get_stats

In [ ]:
def get_stats(dataset):
    """
    Calculate statistics for each column in a given dataset.

    Parameters
    ----------
    dataset : SlewDataSet
        An instance of the dataset class containing the data to be analyzed.

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing calculated statistics for each column in the dataset.
        For each column, the statistics include minimum, maximum, and peak-to-peak values.

    Notes
    -----
    This function computes statistics for each column in the provided dataset. It utilizes the `get_minmax` function
    to calculate minimum, maximum, and peak-to-peak values for each column's data.
    """

    stats = pd.DataFrame(
        data=[get_minmax(dataset.df[col]) for col in dataset.df.columns],
        index=dataset.df.columns
    )
    return stats

#### get_tma_slew_events

In [ ]:
def get_tma_slew_events(start : str, end : str, client : EfdClient | None = None):
    """
    Retrieve Telescope Mount Assembly (TMA) slew events within a specified time range.

    Parameters
    ----------
    start : str
        The start time of the time range in UTC ISO 8601 format.
    end : str
        The end time of the time range in UTC ISO 8601 format.
    client : EfdClient | None, optional
        An instance of EfdClient for connecting to the Event Forwarding Database. Default is None.

    Returns
    -------
    list
        A list of TMA slew events that occurred within the specified time range.

    Notes
    -----
    This function retrieves TMA slew events occurring between the specified start and end times.
    It uses the TMAEventMaker class to obtain events for the specified day of observation (dayObs).
    The events are filtered to include only those that start after 1 second before the specified start time
    and end before 1 second after the specified end time.

    Example
    -------
    >>> start_time = "2023-08-09T00:00:00"
    >>> end_time = "2023-08-09T23:59:59"
    >>> tma_events = get_tma_slew_events(start_time, end_time)
    """
    start = Time(start, scale='utc')
    end = Time(end, scale='utc')
    
    dayObs = int(start.strftime("%Y%m%d"))
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)
    events = [e for e in events if (e.begin > start - TimeDelta(1, format="sec")) 
              & ((e.begin < end + TimeDelta(1, format="sec")))]
    
    return events

## Data Collection

In [ ]:
# Dataset 1 - Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
start = "2023-08-02T22:02:30"
end = "2023-08-02T22:04:00"

dataset1 = SlewDataSet(
    name="Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On",
    start=start, 
    end=end
)

dataset1.df = await query_dataset(dataset1.start, dataset1.end)
dataset1.stats = get_stats(dataset1)

In [ ]:
# Dataset 2 - Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation On
start = "2023-07-28T02:15:45"
end = "2023-07-28T02:17:15"

dataset2 = SlewDataSet(
    name="Az Slew from -100 to 100 with 80 El, 30% motion settings and Inertia Compensation Off",
    start=start, 
    end=end
)

dataset2.df = await query_dataset(dataset2.start, dataset2.end)
dataset2.stats = get_stats(dataset2)

## Plots and Analysis

In [ ]:
def plot_hp_measured_data(df):
    """
    Plots the HP Measured Data as a function of the time.
    
    Parameters
    ----------
    df : pd.DataFrame
        Table containing the HP measured forces (the columns should be 
        `measuredForces0`, `measuredForces1`, etc.). The index should be 
        a timestamp. 
    """                   
    fig, ax = plt.subplots(
        num="HP Measured Forces",
        dpi=120,
        figsize=(12, 4)
    )

    for hp in range(number_of_hardpoints):
        topic = measured_forces_topics[hp]

        ax.plot(
            df[topic], 
            "-",
            label=f"HP{hp+1}",
        )

    ax.set_title(f"HP Measured Data\n {df.index[0]} - {df.index[-1]}")
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel("HP Measured Forces [N]")
    ax.grid(":", alpha=0.2)
    ax.legend(ncol=3)

    plt.show()

In [ ]:
plot_hp_measured_data(dataset1.df)
plot_hp_measured_data(dataset2.df)

In [ ]:
dataset1.stats

In [ ]:
dataset2.stats